In [56]:
import requests
import pandas as pd
import csv
import requests
import sqlite3
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

In [57]:
# Set the API endpoint and parameters
url = 'https://www.alphavantage.co/query'
function1 = 'BALANCE_SHEET'
function2 = 'INCOME_STATEMENT'
symbols = ['PRU', 'MET']
apikey = 'P249K0Q2QUE0A029'
params = {
    'function': function1,
    'apikey': apikey,
}

In [58]:
# Loop through each symbol and retrieve the income statement data
dfs = []
for symbol in symbols:
    params['symbol'] = symbol
    response = requests.get(url, params=params)
    data = response.json()
    balance_sheet_data = data['annualReports']
    df = pd.DataFrame.from_dict(balance_sheet_data)
    df['symbol'] = symbol
    dfs.append(df)

In [59]:
# Combine the data for both symbols into a single DataFrame
df_combined = pd.concat(dfs)

In [60]:
params = {
    'function': function2,
    'apikey': apikey,
}

In [61]:
dfs2 = []
for symbol in symbols:
    params['symbol'] = symbol
    response = requests.get(url, params=params)
    data = response.json()
    income_statement_data = data['annualReports']
    df = pd.DataFrame.from_dict(income_statement_data)
    df['symbol'] = symbol
    dfs2.append(df)

In [62]:
df2_combined = pd.concat(dfs2)

In [63]:
df_merged = pd.merge(df_combined, df2_combined, on=["fiscalDateEnding","symbol"], how='inner')

In [64]:
print(df_merged.shape)

(10, 64)


In [65]:
df_merged.to_csv('PRU_MET.csv', index=False)

In [66]:
# Load the existing workbook
workbook = openpyxl.load_workbook(filename="data_source.xlsx")

In [67]:
# Get the "Company_data" sheet
worksheet = workbook["Company_data"]

In [68]:
# Merge the data into the sheet
df_1 = pd.concat([pd.read_excel("data_source.xlsx", sheet_name="Company_data"), df_merged], ignore_index=True, sort=False)
for r in dataframe_to_rows(df_1, index=False, header=False):
    row_data = []
    for cell in r:
        if cell is not None:
            row_data.append(cell)
        else:
            row_data.append("")
    worksheet.append(row_data)

In [69]:
# Save the changes to the workbook
workbook.save("data_source.xlsx")

In [70]:
# Load the workbook
workbook = openpyxl.load_workbook('data_source.xlsx')

In [71]:
# Select the "Sheet_1" tab
sheet = workbook['Company_data']

# Delete rows 2 to 12
sheet.delete_rows(2, 12)

In [72]:
# Save the workbook
workbook.save('data_source.xlsx')